# Twin Delayed Deep Deterministic Policy Gradient (TD3)

# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter

import gym
import roboschool
import sys

# Networks

In [2]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [3]:
class Actor(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            action output of network with tanh activation
    """
    
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)

        self.max_action = max_action


    def forward(self, x):
        print(x)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.max_action * torch.tanh(self.l3(x)) 
        return x



In [4]:
class Critic(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            value output of network 
    """
    
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)

        # Q2 architecture
        self.l4 = nn.Linear(state_dim + action_dim, 400)
        self.l5 = nn.Linear(400, 300)
        self.l6 = nn.Linear(300, 1)


    def forward(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)

        x2 = F.relu(self.l4(xu))
        x2 = F.relu(self.l5(x2))
        x2 = self.l6(x2)
        return x1, x2


    def Q1(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)
        return x1

# Memory

In [5]:
# Code based on: 
# https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py

# Expects tuples of (state, next_state, action, reward, done)
class ReplayBuffer(object):
    """Buffer to store tuples of experience replay"""
    
    def __init__(self, max_size=1000000):
        """
        Args:
            max_size (int): total amount of tuples to store
        """
        
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, data):
        """Add experience tuples to buffer
        
        Args:
            data (tuple): experience replay tuple
        """
        
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        """Samples a random amount of experiences from buffer of batch size
        
        Args:
            batch_size (int): size of sample
        """
        
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, actions, next_states, rewards, dones = [], [], [], [], []

        for i in ind: 
            s, a, s_, r, d = self.storage[i]
            states.append(np.array(s, copy=False))
            actions.append(np.array(a, copy=False))
            next_states.append(np.array(s_, copy=False))
            rewards.append(np.array(r, copy=False))
            dones.append(np.array(d, copy=False))

        return np.array(states), np.array(actions), np.array(next_states), np.array(rewards).reshape(-1, 1), np.array(dones).reshape(-1, 1)

# Agent

In [6]:
class TD3(object):
    """Agent class that handles the training of the networks and provides outputs as actions
    
        Args:
            state_dim (int): state size
            action_dim (int): action size
            max_action (float): highest action to take
            device (device): cuda or cpu to process tensors
            env (env): gym environment to use
    
    """
    
    def __init__(self, state_dim, action_dim, max_action, env):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=1e-3)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=1e-3)

        self.max_action = max_action
        self.env = env


        
    def select_action(self, state, noise=0.1):
        """Select an appropriate action from the agent policy
        
            Args:
                state (array): current state of environment
                noise (float): how much noise to add to acitons
                
            Returns:
                action (float): action clipped within action range
        
        """
        
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        print(state)
        
        action = self.actor(state).cpu().data.numpy().flatten()
        if noise != 0: 
            action = (action + np.random.normal(0, noise, size=self.env.action_space.shape[0]))
            
        return action.clip(self.env.action_space.low, self.env.action_space.high)

    
    def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
        """Train and update actor and critic networks
        
            Args:
                replay_buffer (ReplayBuffer): buffer for experience replay
                iterations (int): how many times to run training
                batch_size(int): batch size to sample from replay buffer
                discount (float): discount factor
                tau (float): soft update for main networks to target networks
                
            Return:
                actor_loss (float): loss from actor network
                critic_loss (float): loss from critic network
        
        """
        
        for it in range(iterations):

            # Sample replay buffer 
            x, y, u, r, d = replay_buffer.sample(batch_size)
            state = torch.FloatTensor(x).to(device)
            action = torch.FloatTensor(u).to(device)
            next_state = torch.FloatTensor(y).to(device)
            done = torch.FloatTensor(1 - d).to(device)
            reward = torch.FloatTensor(r).to(device)

            # Select action according to policy and add clipped noise 
            noise = torch.FloatTensor(u).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (done * discount * target_Q).detach()

            # Get current Q estimates
            current_Q1, current_Q2 = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q) 

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Delayed policy updates
            if it % policy_freq == 0:

                # Compute actor loss
                actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

                # Optimize the actor 
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

                # Update the frozen target models
                for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)


    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
        torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))


    def load(self, filename="best_avg", directory="./saves"):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
        self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

# Runner

In [7]:
class Runner():
    """Carries out the environment steps and adds experiences to memory"""
    
    def __init__(self, env, agent, replay_buffer):
        
        self.env = env
        self.agent = agent
        self.replay_buffer = replay_buffer
        self.obs = env.reset()
        self.done = False
        
    def next_step(self, episode_timesteps, noise=0.1):
        
        action = self.agent.select_action(np.array(self.obs), noise=0.1)
        
        # Perform action
        new_obs, reward, done, _ = self.env.step(action) 
        done_bool = 0 if episode_timesteps + 1 == 200 else float(done)
    
        # Store data in replay buffer
        replay_buffer.add((self.obs, new_obs, action, reward, done_bool))
        
        self.obs = new_obs
        
        if done:
            self.obs = self.env.reset()
            done = False
            
            return reward, True
        
        return reward, done

# Evaluate

In [8]:
def evaluate_policy(policy, env, eval_episodes=100,render=False):
    """run several episodes using the best agent policy
        
        Args:
            policy (agent): agent to evaluate
            env (env): gym environment
            eval_episodes (int): how many test episodes to run
            render (bool): show training
        
        Returns:
            avg_reward (float): average reward over the number of evaluations
    
    """
    
    avg_reward = 0.
    for i in range(eval_episodes):
        obs = env.reset()
        done = False
        while not done:
            if render:
                env.render()
            action = policy.select_action(np.array(obs), noise=0)
            obs, reward, done, _ = env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("\n---------------------------------------")
    print("Evaluation over {:d} episodes: {:f}" .format(eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward

# Observation

In [9]:
def observe(env,replay_buffer, observation_steps):
    """run episodes while taking random actions and filling replay_buffer
    
        Args:
            env (env): gym environment
            replay_buffer(ReplayBuffer): buffer to store experience replay
            observation_steps (int): how many steps to observe for
    
    """
    
    time_steps = 0
    obs = env.reset()
    done = False

    while time_steps < observation_steps:
        action = env.action_space.sample()
        new_obs, reward, done, _ = env.step(action)

        replay_buffer.add((obs, new_obs, action, reward, done))

        obs = new_obs
        time_steps += 1

        if done:
            obs = env.reset()
            done = False

        print("\rPopulating Buffer {}/{}.".format(time_steps, observation_steps), end="")
        sys.stdout.flush()

# Train

In [10]:
def train(agent, test_env):
    """Train the agent for exploration steps
    
        Args:
            agent (Agent): agent to use
            env (environment): gym environment
            writer (SummaryWriter): tensorboard writer
            exploration (int): how many training steps to run
    
    """

    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    episode_reward = 0
    episode_timesteps = 0
    done = False 
    obs = env.reset()
    evaluations = []
    rewards = []
    best_avg = -2000
    
    writer = SummaryWriter(comment="-TD3_Baseline_HalfCheetah")
    
    while total_timesteps < EXPLORATION:
    
        if done: 

            if total_timesteps != 0: 
                rewards.append(episode_reward)
                avg_reward = np.mean(rewards[-100:])
                
                writer.add_scalar("avg_reward", avg_reward, total_timesteps)
                writer.add_scalar("reward_step", reward, total_timesteps)
                writer.add_scalar("episode_reward", episode_reward, total_timesteps)
                
                if best_avg < avg_reward:
                    best_avg = avg_reward
                    print("saving best model....\n")
                    agent.save("best_avg","saves")

                print("\rTotal T: {:d} Episode Num: {:d} Reward: {:f} Avg Reward: {:f}".format(
                    total_timesteps, episode_num, episode_reward, avg_reward), end="")
                sys.stdout.flush()


                if avg_reward >= REWARD_THRESH:
                    break

                agent.train(replay_buffer, episode_timesteps, BATCH_SIZE, GAMMA, TAU, NOISE, NOISE_CLIP, POLICY_FREQUENCY)

                # Evaluate episode
#                 if timesteps_since_eval >= EVAL_FREQUENCY:
#                     timesteps_since_eval %= EVAL_FREQUENCY
#                     eval_reward = evaluate_policy(agent, test_env)
#                     evaluations.append(avg_reward)
#                     writer.add_scalar("eval_reward", eval_reward, total_timesteps)

#                     if best_avg < eval_reward:
#                         best_avg = eval_reward
#                         print("saving best model....\n")
#                         agent.save("best_avg","saves")

                episode_reward = 0
                episode_timesteps = 0
                episode_num += 1 

        reward, done = runner.next_step(episode_timesteps)
        episode_reward += reward

        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1

# Config

In [11]:
ENV = "Pendulum-v0"#"RoboschoolHalfCheetah-v1"
SEED = 0
OBSERVATION = 10000
EXPLORATION = 5000000
BATCH_SIZE = 100
GAMMA = 0.99
TAU = 0.005
NOISE = 0.2
NOISE_CLIP = 0.5
EXPLORE_NOISE = 0.1
POLICY_FREQUENCY = 2
EVAL_FREQUENCY = 5000
REWARD_THRESH = 8000

# Main

In [12]:
env = gym.make(ENV)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds
env.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
max_action = float(env.action_space.high[0])

policy = TD3(state_dim, action_dim, max_action, env)

replay_buffer = ReplayBuffer()

runner = Runner(env, policy, replay_buffer)

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True

In [16]:
# Populate replay buffer
observe(env, replay_buffer, OBSERVATION)

Populating Buffer 1905/10000.

KeyboardInterrupt: 

In [13]:
# Train agent
train(policy, env)

tensor([[-0.9422, -0.3350,  0.9308]])
tensor([[-0.9422, -0.3350,  0.9308]])
tensor([[0.6663, 0.7457, 0.1572]])
tensor([[0.6663, 0.7457, 0.1572]])
tensor([[0.6391, 0.7691, 0.7185]])
tensor([[0.6391, 0.7691, 0.7185]])
tensor([[0.5874, 0.8093, 1.3080]])
tensor([[0.5874, 0.8093, 1.3080]])
tensor([[0.5060, 0.8626, 1.9479]])
tensor([[0.5060, 0.8626, 1.9479]])
tensor([[0.3886, 0.9214, 2.6272]])
tensor([[0.3886, 0.9214, 2.6272]])
tensor([[0.2313, 0.9729, 3.3149]])
tensor([[0.2313, 0.9729, 3.3149]])
tensor([[0.0296, 0.9996, 4.0746]])
tensor([[0.0296, 0.9996, 4.0746]])
tensor([[-0.2109,  0.9775,  4.8428]])
tensor([[-0.2109,  0.9775,  4.8428]])
tensor([[-0.4728,  0.8811,  5.6006]])
tensor([[-0.4728,  0.8811,  5.6006]])
tensor([[-0.7225,  0.6913,  6.2991]])
tensor([[-0.7225,  0.6913,  6.2991]])
tensor([[-0.9129,  0.4082,  6.8559]])
tensor([[-0.9129,  0.4082,  6.8559]])
tensor([[-0.9982,  0.0593,  7.2239]])
tensor([[-0.9982,  0.0593,  7.2239]])
tensor([[-0.9533, -0.3020,  7.3227]])
tensor([[-0.9533

tensor([[ 0.9919, -0.1273,  0.2204]])
tensor([[ 0.9919, -0.1273,  0.2204]])
tensor([[ 0.9926, -0.1215,  0.1165]])
tensor([[ 0.9926, -0.1215,  0.1165]])
tensor([[ 0.9927, -0.1205,  0.0206]])
tensor([[ 0.9927, -0.1205,  0.0206]])
tensor([[ 0.9924, -0.1230, -0.0499]])
tensor([[ 0.9924, -0.1230, -0.0499]])
tensor([[ 0.9914, -0.1309, -0.1606]])
tensor([[ 0.9914, -0.1309, -0.1606]])
tensor([[ 0.9895, -0.1448, -0.2807]])
tensor([[ 0.9895, -0.1448, -0.2807]])
tensor([[ 0.9863, -0.1647, -0.4018]])
tensor([[ 0.9863, -0.1647, -0.4018]])
tensor([[ 0.9815, -0.1916, -0.5470]])
tensor([[ 0.9815, -0.1916, -0.5470]])
tensor([[ 0.9743, -0.2251, -0.6856]])
tensor([[ 0.9743, -0.2251, -0.6856]])
tensor([[ 0.9635, -0.2678, -0.8817]])
tensor([[ 0.9635, -0.2678, -0.8817]])
tensor([[ 0.9471, -0.3208, -1.1088]])
tensor([[ 0.9471, -0.3208, -1.1088]])
tensor([[ 0.9231, -0.3845, -1.3621]])
tensor([[ 0.9231, -0.3845, -1.3621]])
tensor([[ 0.8880, -0.4598, -1.6623]])
tensor([[ 0.8880, -0.4598, -1.6623]])
tensor([[ 0.

tensor([[ 9.0336e-01, -4.2888e-01,  3.1030e-01],
        [ 7.5748e-01,  6.5286e-01,  5.7861e-01],
        [ 7.2149e-01,  6.9242e-01,  1.0698e+00],
        [-9.5330e-01, -3.0201e-01,  7.3227e+00],
        [ 9.8426e-01, -1.7674e-01,  5.7855e-01],
        [ 7.9042e-01, -6.1257e-01,  2.4797e+00],
        [ 5.8302e-01, -8.1246e-01,  3.3979e+00],
        [-5.6940e-01, -8.2206e-01,  6.9235e+00],
        [ 7.9885e-01,  6.0153e-01, -1.2769e+00],
        [-8.3822e-01,  5.4533e-01, -7.3941e+00],
        [ 9.1863e-01, -3.9512e-01, -1.0283e+00],
        [-8.3822e-01,  5.4533e-01, -7.3941e+00],
        [-5.3992e-01, -8.4172e-01,  6.7310e+00],
        [ 9.6166e-01, -2.7423e-01, -5.5923e-02],
        [ 7.0859e-01, -7.0562e-01,  3.0030e+00],
        [ 9.7433e-01, -2.2511e-01, -6.8556e-01],
        [ 7.0878e-01, -7.0543e-01, -2.5157e+00],
        [ 3.6487e-01,  9.3106e-01, -3.9751e+00],
        [-3.9768e-01, -9.1752e-01,  6.6527e+00],
        [ 9.0299e-01, -4.2966e-01, -1.7332e-02],
        [ 3.4363e-01

tensor([[ 0.6098,  0.7926, -2.9612],
        [-0.4399,  0.8981, -6.7313],
        [-0.8534, -0.5212, -7.0964],
        [ 0.9598, -0.2806,  1.0358],
        [ 0.8408, -0.5413,  2.3136],
        [ 0.9815, -0.1916, -0.5470],
        [ 0.8483, -0.5295,  2.0261],
        [-0.9799, -0.1997, -7.5383],
        [ 0.9440, -0.3299,  1.2700],
        [ 0.3886,  0.9214,  2.6272],
        [-0.2600, -0.9656,  6.1472],
        [ 0.8809, -0.4733, -0.6517],
        [-0.9382, -0.3460, -7.4767],
        [ 0.9895, -0.1448, -0.2807],
        [ 0.8204, -0.5718,  1.7833],
        [ 0.5087,  0.8610, -3.2029],
        [-0.8382,  0.5453, -7.3941],
        [ 0.8236,  0.5672, -0.8453],
        [-0.6755, -0.7374,  7.1526],
        [ 0.9895, -0.1448, -0.2807],
        [-0.5634,  0.8262,  6.0629],
        [ 0.5144,  0.8575,  2.4286],
        [-0.9951, -0.0984,  7.4818],
        [ 0.5933, -0.8049, -3.0511],
        [ 0.8349,  0.5504, -0.4066],
        [ 0.5087,  0.8610, -3.2029],
        [-0.9936,  0.1133,  7.5469],
 

tensor([[ 9.4715e-01, -3.2081e-01, -1.1088e+00],
        [ 9.8148e-01, -1.9158e-01, -5.4703e-01],
        [ 2.4178e-01, -9.7033e-01,  4.9562e+00],
        [-3.6135e-03, -9.9999e-01,  5.6905e+00],
        [ 7.3983e-01,  6.7279e-01, -1.3574e+00],
        [ 9.6057e-01, -2.7802e-01,  3.1530e-01],
        [ 9.9241e-01, -1.2296e-01, -4.9937e-02],
        [ 1.0482e-01,  9.9449e-01,  4.1362e+00],
        [ 3.7075e-01,  9.2873e-01, -3.9647e+00],
        [ 8.0205e-01,  5.9726e-01,  7.9334e-01],
        [-8.3822e-01,  5.4533e-01, -7.3941e+00],
        [-9.3822e-01, -3.4603e-01, -7.4767e+00],
        [ 9.4402e-01, -3.2989e-01,  1.2700e+00],
        [ 6.1211e-01,  7.9078e-01, -2.5016e+00],
        [-7.2254e-01,  6.9133e-01,  6.2991e+00],
        [ 9.8148e-01, -1.9158e-01, -5.4703e-01],
        [ 9.1863e-01, -3.9512e-01, -1.0283e+00],
        [ 5.5983e-01, -8.2861e-01, -2.9752e+00],
        [-5.6940e-01, -8.2206e-01,  6.9235e+00],
        [-7.4706e-01, -6.6476e-01, -7.0079e+00],
        [-5.2910e-02

...

tensor([[-9.9902e-01, -4.4282e-02, -2.1517e-02],
        [ 5.9507e-01, -8.0367e-01,  3.6012e+00],
        [-9.9873e-01, -5.0418e-02,  2.7290e-02],
        [-9.9776e-01, -6.6910e-02,  4.7642e-02],
        [-9.9891e-01, -4.6605e-02, -6.4271e-03],
        [-9.6229e-01,  2.7202e-01,  7.2244e+00],
        [ 9.7433e-01, -2.2511e-01, -6.8556e-01],
        [ 5.8744e-01,  8.0927e-01,  1.3080e+00],
        [-5.2910e-02, -9.9860e-01, -5.0302e+00],
        [-9.9879e-01, -4.9145e-02,  2.5047e-02],
        [-5.6940e-01, -8.2206e-01,  6.9235e+00],
        [-2.5996e-01, -9.6562e-01,  6.1472e+00],
        [ 8.9070e-01, -4.5460e-01, -1.3144e+00],
        [ 7.9168e-01, -6.1094e-01, -2.0480e+00],
        [-9.9999e-01,  5.1564e-03,  1.8439e-01],
        [ 1.5464e-01, -9.8797e-01,  5.2884e+00],
        [ 3.8862e-01,  9.2140e-01,  2.6272e+00],
        [-6.0272e-01,  7.9796e-01, -6.9422e+00],
        [ 9.4830e-01, -3.1737e-01,  7.6292e-01],
        [-9.9852e-01, -5.4377e-02, -3.1329e-02],
        [-9.9877e-01

tensor([[ 7.5358e-01,  6.5736e-01,  2.5704e-01],
        [-9.9891e-01, -4.6605e-02, -6.4271e-03],
        [-9.9867e-01, -5.1616e-02, -7.3405e-01],
        [-9.9749e-01, -7.0800e-02, -8.2718e-05],
        [-9.9907e-01, -4.3075e-02,  2.8178e-02],
        [-9.9852e-01, -5.4370e-02, -2.5760e-02],
        [-9.9944e-01, -3.3496e-02, -1.9385e-02],
        [ 6.6457e-01,  7.4723e-01,  1.5807e+00],
        [-8.0626e-01, -5.9156e-01, -2.5069e-01],
        [-9.9972e-01,  2.3766e-02,  1.6359e-01],
        [-9.9933e-01, -3.6649e-02,  4.1137e-02],
        [-5.4880e-02,  9.9849e-01, -5.5460e+00],
        [ 8.0205e-01,  5.9726e-01,  7.9334e-01],
        [-9.9902e-01, -4.4282e-02, -2.1517e-02],
        [-9.8363e-01, -1.8019e-01, -1.1028e+00],
        [-2.9473e-02,  9.9957e-01,  4.5532e+00],
        [-6.1656e-01,  7.8730e-01, -7.0142e+00],
        [-9.9837e-01, -5.7055e-02,  1.1561e-02],
        [-1.7437e-01, -9.8468e-01, -5.6362e+00],
        [-9.9764e-01, -6.8652e-02,  3.4918e-02],
        [ 8.9660e-01

tensor([[-9.1287e-01,  4.0824e-01,  6.8559e+00],
        [-9.9909e-01, -4.2587e-02, -7.9551e-03],
        [-9.9935e-01, -3.5956e-02, -2.8656e-02],
        [-9.9911e-01, -4.2213e-02, -4.1939e-02],
        [-9.9890e-01, -4.6868e-02,  1.6201e-02],
        [-9.9944e-01, -3.3452e-02,  2.1357e-02],
        [ 9.6057e-01, -2.7802e-01,  3.1530e-01],
        [ 7.9168e-01, -6.1094e-01, -2.0480e+00],
        [-9.9935e-01, -3.5956e-02, -2.8656e-02],
        [ 9.0299e-01, -4.2966e-01, -1.7332e-02],
        [-9.9999e-01,  4.7534e-03, -5.0672e-01],
        [ 5.0867e-01,  8.6096e-01, -3.2029e+00],
        [-9.9911e-01, -4.2205e-02, -4.1581e-02],
        [ 8.5567e-01, -5.1752e-01, -1.0190e+00],
        [-9.9908e-01, -4.2984e-02,  7.3337e-04],
        [-6.0272e-01,  7.9796e-01, -6.9422e+00],
        [ 6.2062e-01,  7.8411e-01,  2.2561e+00],
        [-5.9431e-01, -8.0424e-01, -6.5147e+00],
        [-4.3989e-01,  8.9805e-01, -6.7313e+00],
        [-9.9776e-01, -6.6910e-02,  4.7642e-02],
        [-9.9871e-01

tensor([[ 0.1496, -0.9888, -2.4781]])
tensor([[ 0.1496, -0.9888, -2.4781]])
tensor([[ 2.5821e-03, -1.0000e+00, -2.9510e+00]])
tensor([[ 2.5821e-03, -1.0000e+00, -2.9510e+00]])
tensor([[-0.1667, -0.9860, -3.4014]])
tensor([[-0.1667, -0.9860, -3.4014]])
tensor([[-0.3520, -0.9360, -3.8447]])
tensor([[-0.3520, -0.9360, -3.8447]])
tensor([[-0.5414, -0.8408, -4.2467]])
tensor([[-0.5414, -0.8408, -4.2467]])
tensor([[-0.7180, -0.6960, -4.5773]])
tensor([[-0.7180, -0.6960, -4.5773]])
tensor([[-0.8628, -0.5055, -4.7993]])
tensor([[-0.8628, -0.5055, -4.7993]])
tensor([[-0.9595, -0.2816, -4.8887]])
tensor([[-0.9595, -0.2816, -4.8887]])
tensor([[-0.9990, -0.0452, -4.8058]])
tensor([[-0.9990, -0.0452, -4.8058]])
tensor([[-0.9835,  0.1808, -4.5397]])
tensor([[-0.9835,  0.1808, -4.5397]])
tensor([[-0.9259,  0.3777, -4.1104]])
tensor([[-0.9259,  0.3777, -4.1104]])
tensor([[-0.8446,  0.5354, -3.5546]])
tensor([[-0.8446,  0.5354, -3.5546]])
tensor([[-0.7599,  0.6501, -2.8530]])
tensor([[-0.7599,  0.6501,

KeyboardInterrupt: 

In [ ]:
policy.load()

for i in range(100):
    evaluate_policy(policy, env, render=True)

In [ ]:
env.close()